<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

<img src = "https://github.com/Jangrae/img/blob/master/medical2.png?raw=true" width=800 align="left"/>

# 실습 내용

- 다양한 알고리즘으로 모델을 만들고 성능을 검증합니다.
- 성능이 좋을 것으로 판단된 모델로 예측을 수행합니다.
- 예측된 결과를 기존 평가 파일에 병합합니다.

# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [ ]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'

In [ ]:
# 학습 데이터 불러오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/insurance_train.csv'
data1 = pd.read_csv(path)

# 평가 데이터 불러오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/insurance_test.csv'
data2 = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

In [ ]:
# 학습 데이터 살펴보기
data1.head()

**데이터 설명**

- age: 나이
- sex: 성별(female, male)
- bmi: 체질량지수(체중을 키의 제곱으로 나눈 값, 적정수준:18.5 - 24.9)
- children: 자녀 수
- smoker: 흡연 여부
- region: 거주지역(northeast, southeast, southwest, northwest)
- charges: 건강보험에서 지불한 의료비 - Target

In [ ]:
# 평가 데이터 살펴보기
data2.head()

In [ ]:
# 학습 데이터 기술통계 확인
data1.describe()

In [ ]:
# 평가 데이터 기술통계 확인
data2.describe()

In [ ]:
# 학습 데이터 Target 분포 시각화
plt.figure(figsize=(8, 4))
plt.subplot(2, 1, 1)
plt.hist(data1['charges'], bins=30, alpha=0.7, ec='k')
plt.subplot(2, 1, 2)
plt.boxplot(x=data1['charges'], vert=False)
plt.show()

In [ ]:
# 학습 데이터 흡연자 비율 시각화
plt.figure(figsize=(8, 3))
sns.countplot(x=data1['smoker'])
plt.show()

In [ ]:
# 학습 데이터 남녀 비율 시각화
plt.figure(figsize=(8, 3))
sns.countplot(x=data1['sex'])
plt.show()

In [ ]:
# 학습 데이터 흡연 여부에 따른 의료비 시각화
plt.figure(figsize=(8, 3))
sns.histplot(x=data1['charges'], hue=data1['smoker'], bins=40)
plt.show()

# 3.데이터 준비

- 학습 데이터를 전처리해 모델링을 진행할 수 있는 형태의 데이터를 준비합니다.

**1) x, y 분리**

- Target 변수를 선정하고, x, y로 분리합니다.

In [ ]:
# target 확인
target = 'charges'

# 데이터 분리
x = data1.drop(target, axis=1)
y = data1.loc[:, target]

**2) 가변수화**

- 가변수화를 수행합니다.
- 문자열 변수에 대해서는 필히 가변수화를 수행합니다.

In [ ]:
# 가변수화 대상
dumm_cols = ['sex', 'smoker', 'region']

# 가변수화
x = pd.get_dummies(x, columns=dumm_cols, drop_first=True, dtype=int)

# 확인
x.head()

**3) 학습용, 검증용 데이터 분리**

- 학습용, 검증용 데이터를 7:3으로 분리합니다.

In [ ]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 데이터 분리
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=1)

**4) 정규화**

- KNN 알고리즘을 위해 정규화를 수행합니다.
- 정규화 결과는 별도 변수로 준비해 둡니다. 예) x_train_s, x_val_s

In [ ]:
# 모듈 불러오기
from sklearn.preprocessing import MinMaxScaler

# 정규화
scaler = MinMaxScaler()
x_train_s = scaler.fit_transform(x_train)
x_val_s = scaler.transform(x_val)

# 4.모델링

- 여러 알고리즘으로 최적의 성능을 갖는 모델을 만들고 성능을 검증합니다.
- 검증된 성능 정보를 수집해 비교하여 가장 좋은 성능을 보이는 모델을 선택합니다.
- 모델 이름은 알고리즘마다 다르게 합니다. 예) model_lr, model_dt,...

In [ ]:
# xgboost 설치
# !pip install xgboost

In [ ]:
# lightgbm 설치
# !pip install lightgbm

- 이후 사용할 함수를 모두 불러옵니다.

In [ ]:
# 불러오기
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score

**1) KNN**

- KNN 알고리즘으로 최선의 하이퍼파라미터를 갖는 모델을 만듭니다.

In [ ]:
# 매개변수 범위('n_neighbors': range(1, 21))
params = {'n_neighbors': range(1, 21)}

# 모델 선언
model_knn = GridSearchCV(KNeighborsRegressor(), params, cv=5)

# 학습
model_knn.fit(x_train_s, y_train)

# 매개변수와 성능 확인
print(model_knn.best_params_)
print(model_knn.best_score_)

- 검증 데이터로 성능을 검증하고, 검증된 성능 정보를 수집해 다른 모델과 비교합니다.

In [ ]:
# 예측하기
y_val_pred = model_knn.predict(x_val_s)

# 검증하기
print(mean_absolute_error(y_val, y_val_pred))
print(r2_score(y_val, y_val_pred))

# 성능정보 수집(R2 Score)
result = {}
result['KNN'] = r2_score(y_val, y_val_pred)
print(result)

**2) Decision Tree**

- Decision Tree 알고리즘으로 최선의 하이퍼파라미터를 갖는 모델을 만듭니다.

In [ ]:
# 매개변수 범위('max_depth': range(1, 11))
params = {'max_depth': range(1, 11)}

# 모델 선언
model_dt = GridSearchCV(DecisionTreeRegressor(), params, cv=5)

# 학습
model_dt.fit(x_train, y_train)

# 매개변수와 성능 확인
print(model_dt.best_params_)
print(model_dt.best_score_)

- 검증 데이터로 성능을 검증하고, 검증된 성능 정보를 수집해 다른 모델과 비교합니다.

In [ ]:
# 예측하기
y_val_pred = model_dt.predict(x_val)

# 검증하기
print(mean_absolute_error(y_val, y_val_pred))
print(r2_score(y_val, y_val_pred))

# 성능정보 수집(R2 Score)
result['Decision Tree'] = r2_score(y_val, y_val_pred)
print(result)

**3) Random Forest**

- Random Forest 알고리즘으로 최선의 하이퍼파라미터를 갖는 모델을 만듭니다.

In [ ]:
# 매개변수 범위('max_depth': range(1, 11))
params = {'max_depth': range(1, 11)}

# 모델 선언
model_rf = GridSearchCV(RandomForestRegressor(), params, cv=5)

# 학습
model_rf.fit(x_train, y_train)

# 매개변수와 성능 확인
print(model_rf.best_params_)
print(model_rf.best_score_)

- 검증 데이터로 성능을 검증하고, 검증된 성능 정보를 수집해 다른 모델과 비교합니다.

In [ ]:
# 예측하기
y_val_pred = model_rf.predict(x_val)

# 검증하기
print(mean_absolute_error(y_val, y_val_pred))
print(r2_score(y_val, y_val_pred))

# 성능정보 수집(R2 Score)
result['Random Forest'] = r2_score(y_val, y_val_pred)
print(result)

**4) XGBoost**

- XGBoost 알고리즘으로 최선의 하이퍼파라미터를 갖는 모델을 만듭니다.

In [ ]:
# 매개변수 범위('max_depth': range(1, 11))
params = {'max_depth': range(1, 11)}

# 모델 선언
model_xgb = GridSearchCV(XGBRegressor(), params, cv=5)

# 학습
model_xgb.fit(x_train, y_train)

# 매개변수와 성능 확인
print(model_xgb.best_params_)
print(model_xgb.best_score_)

- 검증 데이터로 성능을 검증하고, 검증된 성능 정보를 수집해 다른 모델과 비교합니다.

In [ ]:
# 예측하기
y_val_pred = model_xgb.predict(x_val)

# 검증하기
print(mean_absolute_error(y_val, y_val_pred))
print(r2_score(y_val, y_val_pred))

# 성능정보 수집(R2 Score)
result['XGBoost'] = r2_score(y_val, y_val_pred)
print(result)

# 5.성능 비교

- 검증된 각 모델의 성능을 비교합니다.

In [ ]:
# 데이터프레임 만들기
score = pd.DataFrame()
score['Model'] = list(result)
score['R2'] = result.values()

# 확인
score

# 6.최종 예측

- 가장 성능이 좋다고 검증된 모델로 평가 데이터에 대해 예측을 수행합니다.
- 평가 데이터 정답이 없으므로 예측 결과에 대한 평가를 수행할 수는 없습니다.

**1) 평가 데이터 준비**

- 평가 대상 데이터를 준비합니다.

In [ ]:
# 평가 데이터
x_test = data2

# 확인
x_test.head()

**2) 가변수화**

- 학습 데이터와 동일한 변수를 갖도록 가변수화를 수행합니다.

In [ ]:
# 가변수화 대상
dumm_cols = ['sex', 'smoker', 'region']

# 가변수화
x_test = pd.get_dummies(x_test, columns=dumm_cols, drop_first=True, dtype=int)

# 확인
x_test.head()

**3) 예측하기**

- 가장 좋은 성능을 검증 받은 모델로 예측을 수행합니다.

In [ ]:
# 예측하기
y_pred = model_rf.predict(x_test)

In [ ]:
# 예측값 확인
print(y_pred[:10])

**4) 예측 결과 병합**

- 예측된 결과를 평가 데이터 파일에 새로운 열로 추가합니다.

In [ ]:
# 예측 결과 병합
data2['prd_charges'] = y_pred

# 롹인
data2.head()

In [ ]:
# 파일 저장
data2.to_excel('insurance_predicted.xlsx', index=False)